# Mission - Classifiez automatiquement des informations

Vous êtes mandaté en tant que Consultant Data Scientist par le département RH de votre client. Il s’agit de l'ESN TechNova Partners, spécialisée dans le conseil en transformation digitale et la vente d’applications en SaaS.
 
Ils font face à un turnover plus élevé que d'habitude et ils souhaitent identifier les causes racines potentielles derrière ces démissions.

### Importation des librairies

In [2]:
# Librairies "classiques"
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import plotly.io as pio
from IPython.display import Image, display

# Librairies scikit-learn
from sklearn.model_selection import (
    train_test_split,
    GridSearchCV, 
    cross_validate,
)
from sklearn.metrics import mean_squared_error, r2_score, mean_absolute_error 
from sklearn.inspection import permutation_importance

#Preprocess
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler

#Modèles
from sklearn.dummy import DummyRegressor
from sklearn.linear_model import LinearRegression
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.model_selection import cross_validate

from sklearn.model_selection import train_test_split, cross_validate
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyClassifier

from sklearn.linear_model import LogisticRegression


### On reprend les éléments du précédent notebook

In [3]:
donnees_modelisation = pd.read_csv("../Data/Projet_4_etape2_clean.csv")

# Partie 3 - Réalisation d'un premier modèle de classification

### Recommandation : Commencer par réaliser une séparation train test simple ou une validation croisée simple.
### Recommandation : Commencer par entraîner d’abord un modèle Dummy, puis un modèle linéaire, avant d'entraîner un modèle non-linéaire. Cela vous permettra d’évaluer la difficulté et la “non-linéarité” du jeu de données.

Dans un premier temps, nous allons procéder à une validation croisée sur un modèle Dummy pour avoir une base de comparaison. 

### Test sur le modèle DummyClassifier

DummyClassifier est un classificateur qui fait des prédictions à l'aide de règles simples.

In [9]:
# Colonnes à scaler ou déjà encodées
features_a_scaler = [
    'revenu_mensuel','annee_experience_totale','annees_dans_l_entreprise','distance_domicile_travail',
    'annees_depuis_la_derniere_promotion','experience_externe','score_satisfaction',
    'augmentation_par_formation','pee_par_anciennete','niveau_education']
features_encodees = [
    'genre','heure_supplementaires',
    'frequence_deplacement','a_suivi_formation','tranche_age','statut_marital_Celibataire',
    'statut_marital_Divorce','statut_marital_Marie','promotion_recente','poste_AssistantdeDirection',
    'poste_CadreCommercial','poste_Consultant','poste_DirecteurTechnique','poste_Manager',
    'poste_ReprésentantCommercial','poste_RessourcesHumaines','poste_SeniorManager','poste_TechLead']

# Séparation de X et y
y = donnees_modelisation["a_quitte_l_entreprise"]
X = donnees_modelisation.drop(columns = "a_quitte_l_entreprise")

# Split en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42, stratify=y)

# Scaling ou non de mes features
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), features_a_scaler),
        ('cat', 'passthrough', features_encodees)
    ]
)
# Ajout dans un Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', DummyClassifier(strategy='most_frequent'))
])
# Enregistrement de nos indicateurs
scoring = ['accuracy','precision','recall','f1']

cv_results = cross_validate(
    pipeline,
    X_train, y_train,            
    cv=3,
    scoring=scoring,
    return_train_score=True
)

print("=== Résultats CV (train vs val) ===")
for metric in scoring:
    tr = cv_results[f"train_{metric}"]
    te = cv_results[f"test_{metric}"]
    print(f"{metric:9s}: train {tr.mean():.3f} ± {tr.std():.3f} | val {te.mean():.3f} ± {te.std():.3f}")

=== Résultats CV (train vs val) ===
accuracy : train 0.838 ± 0.001 | val 0.838 ± 0.001
precision: train 0.000 ± 0.000 | val 0.000 ± 0.000
recall   : train 0.000 ± 0.000 | val 0.000 ± 0.000
f1       : train 0.000 ± 0.000 | val 0.000 ± 0.000


/Users/florianschorer/OpenClassrooms Project/Classifiez_automatiquement_des_informations/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/florianschorer/OpenClassrooms Project/Classifiez_automatiquement_des_informations/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", result.shape[0])
/Users/florianschorer/OpenClassrooms Project/Classifiez_automatiquement_des_informations/.venv/lib/python3.12/site-packages/sklearn/metrics/_classification.py:1731: UndefinedMetricWarning: Precision is ill-

Avant d'interpréter nos indicateurs, une petite définition pour chaque.

* Accuracy : C’est le pourcentage de prédictions correctes (tous labels confondus). Exemple : "Parmi tous les employés, combien ai-je bien classés ?”

* Precison : Mesure la fiabilité des prédictions positives. Exemple : “Sur tous ceux que je pense quitter l’entreprise, combien quittent vraiment ?”
* Recall : Mesure la capacité à ne pas rater de cas positifs. Exemple “Sur tous les départs réels, combien ai-je détectés ?”
* F1-score : C’est une moyenne entre Precision et Recall.

* C'est ce qu'on attend d'un modèle de ce type, il va nous servir de base. Nous avons un dataset déséquilibré, 83,8% des salariés sont restés, et nous avons un accuracy de 83,8%, cela correspond.
* Precision / Recall / F1 = 0. Le modèle ne prédit jamais la classe “Oui” → donc impossible de calculer la précision ou le rappel pour cette classe : affichage du warning

### Test sur le modèle LogisticRegression

In [ ]:
# Colonnes à scaler ou déjà encodées
features_a_scaler = [
    'revenu_mensuel','annee_experience_totale','annees_dans_l_entreprise','distance_domicile_travail',
    'annees_depuis_la_derniere_promotion','experience_externe','score_satisfaction',
    'augmentation_par_formation','pee_par_anciennete','niveau_education']
features_encodees = [
    'genre','heure_supplementaires',
    'frequence_deplacement','a_suivi_formation','tranche_age','statut_marital_Celibataire',
    'statut_marital_Divorce','statut_marital_Marie','promotion_recente','poste_AssistantdeDirection',
    'poste_CadreCommercial','poste_Consultant','poste_DirecteurTechnique','poste_Manager',
    'poste_ReprésentantCommercial','poste_RessourcesHumaines','poste_SeniorManager','poste_TechLead']

# Séparation de X et y
y = donnees_modelisation["a_quitte_l_entreprise"]
X = donnees_modelisation.drop(columns = "a_quitte_l_entreprise")

# Split en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20, random_state = 42, stratify=y)

# Scaling ou non de mes features
preprocessor = ColumnTransformer(
    transformers = [
        ('num', StandardScaler(), features_a_scaler ),
        ('cat', 'passthrough', features_encodees)
    ]
)
# Ajout dans un Pipeline
pipeline = Pipeline(steps=[
    ('preprocessor', preprocessor),
    ('model', LogisticRegression())
])
# Enregistrement de nos indicateurs
scoring = ['accuracy','precision','recall','f1']

cv_results = cross_validate(
    pipeline,
    X_train, y_train,            
    cv=3,
    scoring=scoring,
    return_train_score=True # On inclut les scores du train
)

print("=== Résultats CV (train vs val) ===")
for metric in scoring:
    tr = cv_results[f"train_{metric}"]
    te = cv_results[f"test_{metric}"]
    print(f"{metric:9s}: train {tr.mean():.3f} ± {tr.std():.3f} | val {te.mean():.3f} ± {te.std():.3f}")

=== Résultats CV (train vs val) ===
accuracy : train 0.880 ± 0.005 | val 0.881 ± 0.013
precision: train 0.786 ± 0.029 | val 0.791 ± 0.096
recall   : train 0.355 ± 0.034 | val 0.358 ± 0.043
f1       : train 0.488 ± 0.031 | val 0.492 ± 0.059


* Accuracy : train 0.880 vs val 0.881. Le modèle classe correctement environ 88 % des salariés, aussi bien sur train que sur validation. L’écart train/val est quasi nul : pas d’overfitting. On peut noter également un meilleur score d'accuracy pour le modèle de régression logistique que notre étalon Dummy.

* Precision : train 0.786 vs val 0.791. Quand le modèle prédit “Oui” (départ), il a raison environ 8 fois sur 10. Donc peu de faux positifs (il ne se trompe pas souvent en disant qu’un salarié part).
* Recall : train 0.355 vs val 0.358. Mais il ne détecte qu’environ 1 salarié sur 3 qui part réellement. Beaucoup de faux négatifs : le modèle loupe une majorité des départs.
* F1 : train 0.488 | val 0.492. Le compromis précision/rappel est moyen.